# Linux Basics

## Overview:
Faced with a blinking cursor on your command line, Linux can be intimidating to new users who do not know where to start. This lab is designed to show you a series of must-know commands that will prove useful on all facets of system administration, from finding files to checking the status of system resources.

## Goal:
Build a toolbox of basic commands useful to every Linux user.

- Concepts included in this scenario:
- Viewing file contents with cat
- Viewing the last 10 lines of a file with tail
- Appending command outputs to a file with the >> operator
- Locating files that match certain criteria using find and locate
- Using grep with other commands to isolate a portion of their output
- Checking disk space with df
- Viewing and managing processes with ps and top
- Using free to view information about the system memory usage


# Step 1
Manipulating file contents
The first step towards modifying files is knowing which files exist. Use ls to take a look at the contents of the current directory:

```bash
ls
```

There are a few configs and a subdirectory, but you will use quote.txt for this exercise. Use the cat command to quickly check what is in this file. The cat command is short for "concatenate", but it is much more versatile than that name suggests.

```bash
cat quote.txt
```

Just thinking about everything I have been through, and how huge it all feels.
The fact that it is just a small part of something larger
Now you can complete the quote by redirecting the output of the echo command. The command below uses the append redirection operator (>>) to cause the output of the echo command to be written to the end of the file quote.txt.

```bash
echo "makes it seem more manageable." >> quote.txt
```

There is no output, so view the file contents once again to see this change reflected:

```bash
cat quote.txt
```

Just thinking about everything I have been through, and how huge it all feels.
The fact that it is just a small part of something larger
makes it seem more manageable.

The >> operator allows you to add to files from the command line and is particularly useful in shell scripting. You can also use it when a command has a lengthy output and you want to view that output in a text editor.

NOTE: You may also encounter the > redirection operator. This operator will write the output to a file, overwriting the file if it already exists. By contrast, the >> operator will append the output to the specified file. The input operator (<) will redirect data as an input to whatever is called before the operator. The pipe operator (|) is used to send the output of one command to another command so that you can carry out an operation on the results of the first command.

## Step 2
Viewing the end of log files with tail
Some files are large enough that it is not practical to display their contents with cat. A common example is log files, which are often packed full of information.

Since tail only displays the last ten lines of a file, it is particularly useful for viewing recent entries in log files. Take a look at the final ten lines of the boot log.

```bash
tail /var/log/boot.log

Starting Enable periodic update of entitlement certificates....
Starting Dynamic System Tuning Daemon...
Starting Network Manager Wait Online...
Starting Hostname Service...
[  OK  ] Started Enable periodic update of entitlement certificates..
[  OK  ] Started Permit User Sessions.
Starting Hold until boot process finishes up...
Starting Terminate Plymouth Boot Screen...
[  OK  ] Started Command Scheduler.
[  OK  ] Started OpenSSH server daemon.
```

If you wish to then see the entire file, a text viewer like less or view will show the entire file.


Adding the -f option will follow the log file so that you can see new entries. 


```bash
tail -f /var/log/messages

<< OUTPUT ABRIDGED >>
Jul 15 22:37:26 6d5380e16498 NetworkManager[886]:   [1626403046.8294] device (ens5): Activation: failed for connection 'Wired connection 1'
Jul 15 22:37:26 6d5380e16498 NetworkManager[886]:   [1626403046.8298] device (ens5): state change: failed -> disconnected (reason 'none', sys-iface-state: 'managed')
Jul 15 22:37:26 6d5380e16498 NetworkManager[886]:   [1626403046.8343] dhcp4 (ens5): canceled DHCP transaction
Jul 15 22:37:26 6d5380e16498 NetworkManager[886]:   [1626403046.8343] dhcp4 (ens5): state changed timeout -> done
The logger utility will write to this messages log. From the original terminal, write to this log:

logger Hello World
```

Check that this message was recorded by returning to Terminal, where tail is following the contents of this log:

```bash 
Jul 15 22:39:06 6d5380e16498 dnf[3679]: Metadata cache created.
Jul 15 22:39:06 6d5380e16498 systemd[1]: dnf-makecache.service: Succeeded.
Jul 15 22:39:06 6d5380e16498 systemd[1]: Started dnf makecache.
Jul 15 22:42:07 6d5380e16498 root[3693]: Hello World
```

The message you just sent with logger is present in messages.

NOTE: If it took you a bit to switch back to Terminal, there may be additional entries in the messages log below your "Hello World".

Leave this tail -f process running in Terminal, you will use it again in an upcoming step. The next step will walk you through locating files.

## Step 3
Locating files
Manipulating files is only useful if you can locate the file you need. This step will walk you through two commands that are useful for this, locate and find.

The find command is great for searching for files which satisfy some specified criteria. This step will show an example using the filename, but you can also use it to look for files with certain permissions, empty files, or much more.
```bash
find / -name messages

/var/log/messages
```

The output is the full path to the file. If there are multiple matches, then this output would have multiple lines and it would be up to the user to decide which is the appropriate file. Instead of having to search through countless directories, you can quickly locate the file.

find can be input/output intensive, which can sap system resources, especially if you run a search that will scan a large amount of candidate files. In fact, the search above is exactly that kind of search, since it is looking for filenames starting from the root directory. It will check every single file on the entire system to see if it is a match. The locate command instead queries a database to avoid being too I/O intensive.

```bash
locate messages

<< OUTPUT ABRIDGED >>
/usr/share/vim/vim80/syntax/messages.vim
/usr/share/vim/vim80/syntax/msmessages.vim
/var/log/messages
```

This command finds a variety of files containing messages in their name, including the log file that you are looking for, /var/log/messages. The database that locate uses to optimize the search process is updated once per day by default, so that means if you are trying to locate a newly created file it will not be present in the database.

Create a new file with touch:

```bash
touch newFile
```
Try to find this newly created file using locate:

```bash
locate newFile
```

There will be no output, as newFile is not in the locate database. Update the locate database using the updatedb command:

```bash
updatedb
```

Re-run the locate command to look through the newly updated database:

```bash
locate newFile

/root/newFile
```

Now the file is present. This is important to keep in mind so that you do not run into the frustrating situation where locate failed to find a file that existed all along.

## Step 4
Searching for text within files and outputs
find is great if you are looking for criteria about a file, but if you instead want to locate specific file contents, the grep command is what you need. To illustrate this, search for the words you added to the messages log file. The command below uses the -r option to search recursively from the starting directory (/var/log). This will return all locations in any file residing in a subdirectory of /var/log that contain the string "Hello World".

```bash
grep -r "Hello World" /var/log


/var/log/messages:Jul 16 21:38:22 be3f7e9f7264 root[24468]: Hello World
```

The output shows the line you added using logger earlier in this lab. If you do not specify a search location in the grep command, it will default to your current directory. Additionally, the string being searched for is case-sensitive by default. If you want a case-insensitive search, add the -i option.

```bash
grep -ir "hello world" /var/log

/var/log/messages:Jul 16 21:38:22 be3f7e9f7264 root[24468]: Hello World
```

This option can save you from believing that a string does not exist if you weren't sure about how it was capitalized in the file.

Sometimes you may want to see a few more lines before or after the result of your search. This is done using the context option (-C). This option allows you to specify a number of lines before and after the result of the search to display.

```bash
grep -C1 fact quote.txt
```

Just thinking about everything I have been through, and how huge it all feels.
The fact that it is just a small part of something larger
makes it seem more manageable.

NOTE: If you wish to only display lines after the result, use -A (context after). For lines before the result, use -B (context before).

Another way to leverage grep is by piping the output of another command into grep using the | operator. This saves you effort if you are just looking for a specific word or phrase in a lengthy output. You will get a chance to try this out in step 6 of this lab.

## Step 5
Using df to check disk usage
A large part of a system administrator's job is to be able to diagnose what is causing a system to misbehave. Disk space being full can lead to unexpected behavior, so the df command is great to include in any troubleshooting session.

The df command, short for "disk free", shows a breakdown of disk usage. The -h option will present the sizes in a human readable format:
```bash
df -h

Filesystem             Size  Used Avail Use% Mounted on
devtmpfs               890M     0  890M   0% /dev
tmpfs                  909M     0  909M   0% /dev/shm
tmpfs                  909M  8.5M  901M   1% /run
tmpfs                  909M     0  909M   0% /sys/fs/cgroup
/dev/mapper/rhel-root   35G  4.5G   30G  14% /
/dev/vda1             1014M  197M  818M  20% /boot
tmpfs                  182M     0  182M   0% /run/user/0
```

The usage percentage (Use%) column is a great place to start. If any filesystem is close to capacity, then it may be unable to perform installations or other operations that require data to be written. However, the absolute size of the partition will determine what "close to capacity" means. In the case of this system, the root partition is 35 GiB. Therefore, even if it is at 80% usage, the filesystem will still have 7 GiB to work with. Probably something you need to address eventually, but unless you are doing some storage-intensive operation that is failing, likely not the cause of immediate issues.

The boot partition is on the other end of the spectrum, with a size of only 1014 MiB. Therefore, if this filesystem is at 80% usage, there will only be 200 MiB available. That may not be enough for basic operations such as installing system updates. Therefore, it is very concerning if you run df and discover one of the smaller filesystems is at 80% usage.

NOTE: The tmpfs entries are filesystems accessible by the machine that are stored in RAM. This means that you don't have to care as much about these filesystems when troubleshooting using df. A reboot will give you a fresh copy of these filesystems.

## Step 6
Viewing processes
Just as files organize how data is stored on Linux, processes organize all of the programs that are running on the system. The ps command (short for Process Status) will display information about the processes running in the current shell. To see all of the processes running on the system and present them in a user-readable format, it is common to add the aux options to ps.

```bash
ps aux

root           1  0.0  0.7 186400 14448 ?        Ss   15:38   0:11 /usr/lib/systemd/systemd --switched-root --system --deserialize 17
root           2  0.0  0.0      0     0 ?        S    15:38   0:00 [kthreadd]
root           3  0.0  0.0      0     0 ?        I<   15:38   0:00 [rcu_gp]
root           4  0.0  0.0      0     0 ?        I<   15:38   0:00 [rcu_par_gp]
root           6  0.0  0.0      0     0 ?        I<   15:38   0:00 [kworker/0:0H-events_highpri]
root           9  0.0  0.0      0     0 ?        I<   15:38   0:00 [mm_percpu_wq]
root          10  0.0  0.0      0     0 ?        S    15:38   0:00 [ksoftirqd/0]
root          11  0.0  0.0      0     0 ?        I    15:38   0:00 [rcu_sched]

<< OUTPUT ABRIDGED >>
```

The output of this command is often very lengthy, so this is a great chance to apply what you just learned a few steps ago. Using the pipe operator (|), pipe the output of ps aux into the grep command to search for a specific process. Search for the tail -f process that you started in step 2:

```bash
ps aux | grep tail

root       10046  0.0  0.0   7344   928 pts/1    S+   19:26   0:00 tail -f /var/log/messages
root       10051  0.0  0.0  12136  1136 pts/0    S+   19:26   0:00 grep --color=auto tail
```

Here the search finds the tail -f process, but the second line of this output shows that the search also finds itself.

ps is great for showing a snapshot of the active processes and their resource usage, but sometimes it is more useful to have a utility that keeps updating this information in real time. top does just that.

```bash
top

top - 21:11:07 up  4:10,  2 users,  load average: 0.45, 0.16, 0.06
Tasks: 114 total,   1 running, 113 sleeping,   0 stopped,   0 zombie
%Cpu(s):  0.0 us,  0.3 sy,  0.0 ni, 99.3 id,  0.2 wa,  0.2 hi,  0.0 si,  0.0 st
MiB Mem :   1817.0 total,   1135.0 free,    181.5 used,    500.5 buff/cache
MiB Swap:   4000.0 total,   4000.0 free,      0.0 used.   1482.2 avail Mem

  PID USER    PR  NI   VIRT   RES   SHR S %CPU %MEM    TIME+ COMMAND
 2940 root    20  0       0     0     0 I  0.3  0.0  0:00.01 kworker/u4:2-flush-253:0
15893 root    20  0   65416  4736  3952 R  0.3  0.3  0:00.04 top
    1 root    20  0  187008  4968  9800 S  0.0  0.8  0:03.06 systemd
    2 root    20  0       0     0     0 S  0.0  0.0  0:00.01 kthreadd
    3 root     0 -20      0     0     0 I  0.0  0.0  0:00.00 rcu_gp
```

In addition to the process info, top includes some status information:

```bash
The system uptime
Total number of user accounts
1 minute, 5 minute, and 15 minute load averages
```

The total number of tasks and a breakdown of their status counts
The kinds of tasks using the CPU
A breakdown of memory and swap usage

NOTE: For more information on the top preamble, visit section 2 of the top man page.

This output will continue updating until you quit top, making it particularly useful for diagnosing issues that change over time. Leave top running in this terminal for now, you will return to it later.

## Step 7
Using free to view information about the system memory usage
Processes require memory to run, so when the system is running low on free memory it can slow down significantly. The free command is useful for quickly getting a summary of memory usage.

```bash
free -h

The -h option outputs the information in a human-readable format.

           total       used      free    shared   buff/cache  available
Mem:       1.8Gi       156Mi     1.2Gi   16Mi     430Mi       1.5Gi
Swap:      3.9Gi       0B        3.9Gi
```

If the available memory is very low, the fourth column of ps au can be used for finding which processes are the culprits. To sort the output by memory usage, add the --sort=-%mem option:

```bash
ps au --sort=-%mem

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root        1935  0.5  0.2  65416  4704 pts/1    S+   15:05   0:00 top
root        1914  0.1  0.2  27012  4652 pts/1    Ss   15:05   0:00 -bash
<< OUTPUT ABRIDGED >>
```

The first entry on this sorted list belongs to top from the previous step. Suppose you no longer needed top and wanted to free up the resources being used by this process. One option would be to return to terminal that it is running in and quit it. However, it is simpler in most cases to use the kill command. The process ID shown in the ps output is how you refer to the process when killing it. Kill the top process from this terminal:

```bash
kill $(pidof top)
```

This does not show any output, but returning to Terminal 3 reveals that top is no longer updating.

